In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import mpld3
mpld3.enable_notebook()

In [3]:
import sys
sys.path.append('../../Utils/')

In [4]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10

In [5]:
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import nltk
import math
import random
from sklearn.utils import shuffle
from load_imdb_data import load_imdb_data
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np 
import matplotlib.pyplot as plt 
np.random.seed(0)

In [6]:
imdb_data = load_imdb_data()

In [7]:
imdb_data = shuffle(imdb_data)
imdb_data.reset_index(drop=True, inplace=True)

In [8]:
len(imdb_data)

50000

In [9]:
imdb_data.head()

,review,sentiment
0,often tagged as a comedy the man in the white ...,1
1,it s easy to make really general comments abou...,-1
2,i think the movie was one sided i watched it r...,-1
3,i have fond memories of watching this visually...,1
4,this episode had potential the basic premise o...,-1


In [11]:
def processDocs(documents, vocab_size=100000):
    """
    This functions takes in a collection of documents and generates a vocabulary based on the size given in input. 
    It returns a representation for each document in the list of input documents. 
    """
    vocab = {} 
    doc_id = 0 
    doc_ids = []
    
    for doc in documents:
        doc_ids.append(doc_id)                          # Give an ID to each document 
        doc_id += 1
        
        for word in nltk.word_tokenize(doc):            # Generate a vocabulary while iterating threw the documents 
            if word not in vocab:
                vocab[word] = 1 
            else:
                vocab[word] += 1
    
    # Extract the most frequent words based on the vocabulary size 
    freq_words_list = sorted(vocab.items(), key=lambda x: x[1], reverse=True)[:vocab_size]
    freq_words_set = set([item[0] for item in freq_words_list])
    
    # Give an index to each word in vocabulary 
    word2idx = {}         
    index_word = 0
    for word in freq_words_set:
        word2idx[word] = index_word
        index_word += 1
    word2idx['UNK'] = index_word
    
    doc_repr = []                          # Represent each document with representation based on the vocabulary  
    for doc in documents:
        temp = []
        for w in doc:
            if w in word2idx:
                temp.append(word2idx[w])
            else:
                temp.append(word2idx['UNK'])
        doc_repr.append(temp)
        
    return documents, doc_ids, word2idx, doc_repr

In [12]:
docs, doc_ids, word2ids, doc_repr = processDocs(imdb_data['review'])

In [13]:
print(len(docs), len(doc_ids), len(word2ids), len(doc_repr))

(50000, 50000, 88863, 50000)


In [14]:
def performanceTest(X, y, method='tf-idf'):
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y, 
                                                        test_size=0.2,  
                                                        random_state=42)
    
    if method == "tf-idf":
        vectorizer = TfidfVectorizer(min_df=1)
        X_train = vectorizer.fit_transform(X_train)
        X_test = vectorizer.transform(X_test)
        clf = LogisticRegression()
        clf.fit(X_train, y_train)
        train_pred = clf.predict(X_train)
        test_pred = clf.predict(X_test)
        print("Train F1-score : ", f1_score(y_train, train_pred))
        print("Test F1-score : ", f1_score(y_test, test_pred))
    else:
        clf = LogisticRegression()
        clf.fit(X_train, y_train)
        train_pred = clf.predict(X_train)
        test_pred = clf.predict(X_test)
        print("Train F1-score : ", f1_score(y_train, train_pred))
        print("Test F1-score : ", f1_score(y_test, test_pred))

In [15]:
performanceTest(imdb_data['review'], imdb_data['sentiment'])

('Train F1-score : ', 0.75810542821565297)
('Test F1-score : ', 0.65317266051591039)


## Architecture - PV-DM Distributed Memory version of Paragraph Vector

### Basic implementation

##### Initialize tensorflow matrix and matrices for word2vecs

In [ ]:
doc_size = len(docs)
embedding_size_w = 50
embedding_size_d = 50
vocab_size = len(word2ids)
window_size = 5
n_neg_samples = 10
learning_rate = 10e-5
epochs = 201
combined_embed_vector_length = embedding_size_d + embedding_size_w

In [ ]:
# Define placeholders for training 
train_wX = tf.placeholder(tf.int32, shape=[window_size])
train_dX = tf.placeholder(tf.int32, shape=[1])
train_label = tf.placeholder(tf.int32, shape=[None, 1])

In [ ]:
# Define matrix for doc_embedding and word_embedding 
doc_embedding = tf.Variable(tf.random_uniform([doc_size, embedding_size_d], -1.0, 1.0), name="doc_embedding")
word_embedding = tf.Variable(tf.random_uniform([vocab_size, embedding_size_w], -1.0, 1.0), name="word_embedding")

In [ ]:
# Define weights for the output unit 
weights = tf.Variable(tf.truncated_normal([vocab_size, combined_embed_vector_length], 
                                       stddev=1.0 / math.sqrt(combined_embed_vector_length)))
biases = tf.Variable(tf.zeros(vocab_size))

In [ ]:
embed = []

# generating a vector of size embedding_size_d
embed_w = tf.zeros([1, embedding_size_w], dtype=tf.float32)

# add all the word vecs in window_size
for j in range(window_size):
    embed_w += tf.nn.embedding_lookup(word_embedding, train_wX[j])
embed.append(embed_w)

# Add the doc2vec from the doc_embedding 
embed_d = tf.nn.embedding_lookup(doc_embedding, train_dX)
embed.append(embed_d)

embed = tf.concat(1, embed)
embed = tf.reshape(embed, [1, 100])

In [ ]:
loss = tf.nn.sampled_softmax_loss(weights=weights, \
                                  biases=biases, \
                                  labels=train_label, \
                                  inputs=embed, \
                                  num_sampled=n_neg_samples, \
                                  num_classes=vocab_size)

In [ ]:
loss = tf.reduce_mean(loss)

In [ ]:
optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)

In [ ]:
saver = tf.train.Saver()

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    average_loss = 0
    
    for step in range(epochs):
        epoch_error = 0.0
        for id_, repr_ in zip(doc_ids, doc_repr):
            if len(repr_) < window_size + 1:
                continue
            count = random.randint(1, 5)
            for cn in range(count):
                # Mistake here as well 
                sample_window = random.sample(repr_, window_size + 1)

                feed_dict = {train_wX : sample_window[:-1],\
                             train_dX : [id_], \
                             train_label : np.array([sample_window[-1]]).reshape(-1, 1)}
                                
                #print feed_dict
                op, l = sess.run([optimizer, loss], 
                                    feed_dict=feed_dict)
                
                epoch_error += l
                
        if step % 10 == 0:
            print "Error at epoch : ", step, " = ", epoch_error
            
    save_path = saver.save(sess, "./models/model_pvdm.ckpt")
    print("Model saved in file: %s" % save_path)

In [ ]:
doc_pvdm = None

with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "./models/model.ckpt")
    print("Model restored.")
    
    # Normalize word2vec 
    norm_w = tf.sqrt(tf.reduce_sum(tf.square(word_embedding), 1, keep_dims=True))
    normalized_word_embeddings = word_embedding / norm_w
    word2vec = normalized_word_embeddings.eval()
    
    # Normalize doc2vec 
    norm_d = tf.sqrt(tf.reduce_sum(tf.square(doc_embedding), 1, keep_dims=True))
    normalized_doc_embeddings = doc_embedding / norm_d
    
    # Find performance 
    performanceTest(normalized_doc_embeddings.eval(), imdb_data['sentiment'][:1000], method=None)
    doc_pvdm = normalized_doc_embeddings.eval()

### Adding batchsizes for speedup

In [16]:
bucket_list = []

def generate_batch_pvdm(doc_ids, doc_repr, sample_size=5, batch_size=1000, window_size=10):
    global bucket_list

    docs_ids_to_select = list(set(doc_ids) - set(bucket_list))
    
    
    if len(docs_ids_to_select) < batch_size//sample_size:
        bucket_list = []
        docs_ids_to_select = doc_ids
        
    index = 0 
    train_wX = np.ndarray(shape=(batch_size, window_size), dtype=np.int32)
    train_dX = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    train_label = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    random_docs = random.sample(docs_ids_to_select, batch_size//sample_size)    # Choose set of random documents 

    bucket_list += random_docs
    
    for id_ in random_docs:
        for j in range(sample_size):                                 # Generating a dataset of sample size 
            random_index = random.randint(0, len(doc_repr[id_]) - window_size - 1)
            sample_window = doc_repr[id_][random_index: random_index + window_size + 1]
            train_wX[index] = sample_window[:-1]
            train_dX[index] = id_
            train_label[index] = sample_window[-1]  
            index += 1
    return train_wX, train_dX, train_label 

In [28]:
doc_size = len(docs)
embedding_size_w = 100
embedding_size_d = 100
vocab_size = len(word2ids)
window_size = 10
n_neg_samples = 10
learning_rate = 10e-5
epochs = 10001
batch_size=1000
mu=0.99
combined_embed_vector_length = embedding_size_d + embedding_size_w

In [29]:
# Define placeholders for training 
train_wX = tf.placeholder(tf.int32, shape=[batch_size, window_size])
train_dX = tf.placeholder(tf.int32, shape=[batch_size, 1])
train_label = tf.placeholder(tf.int32, shape=[batch_size, 1])

In [30]:
# Define matrix for doc_embedding and word_embedding 
doc_embedding = tf.Variable(tf.random_uniform([doc_size, embedding_size_d],
                                              -np.sqrt(6/(doc_size + embedding_size_d)), 
                                              np.sqrt(6/(doc_size + embedding_size_d))), 
                                              name="doc_embedding")
word_embedding = tf.Variable(tf.random_uniform([vocab_size, embedding_size_w],
                                               -np.sqrt(6/(doc_size + embedding_size_d)),
                                               np.sqrt(6/(doc_size + embedding_size_d)))
                                               ,name="word_embedding")

In [31]:
# Define weights for the output unit 
weights = tf.Variable(tf.truncated_normal([vocab_size, combined_embed_vector_length], 
                                       stddev=1.0 / math.sqrt(combined_embed_vector_length)))
biases = tf.Variable(tf.zeros(vocab_size))

In [32]:
embed = []

# generating a vector of size embedding_size_d
embed_w = tf.zeros([1, embedding_size_w], dtype=tf.float32)

# add all the word vecs in window_size
for j in range(window_size):
    embed_w += tf.nn.embedding_lookup(word_embedding, train_wX[:, j])
embed.append(embed_w)

# Add the doc2vec from the doc_embedding 
embed_d = tf.nn.embedding_lookup(doc_embedding, train_dX[:, 0])
embed.append(embed_d)

print embed_w, embed_d

embed = tf.concat(1, embed)

Tensor("add_19:0", shape=(1000, 100), dtype=float32) Tensor("embedding_lookup_21:0", shape=(1000, 100), dtype=float32)


In [33]:
loss = tf.nn.sampled_softmax_loss(weights=weights, \
                                  biases=biases, \
                                  labels=train_label, \
                                  inputs=embed, \
                                  num_sampled=n_neg_samples, \
                                  num_classes=vocab_size)

In [34]:
loss = tf.reduce_mean(loss)

In [35]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=mu).minimize(loss)
#optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)

"""
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.01
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           1000, 0.96, staircase=True)
# Passing global_step to minimize() will increment it at each step.
optimizer = (
    tf.train.MomentumOptimizer(learning_rate, momentum=mu).minimize(loss, global_step=global_step)
)
"""

'\nglobal_step = tf.Variable(0, trainable=False)\nstarter_learning_rate = 0.01\nlearning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,\n                                           1000, 0.96, staircase=True)\n# Passing global_step to minimize() will increment it at each step.\noptimizer = (\n    tf.train.MomentumOptimizer(learning_rate, momentum=mu).minimize(loss, global_step=global_step)\n)\n'

In [36]:
#optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)

In [37]:
saver = tf.train.Saver()

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    average_loss = 0
    
    for step in range(epochs):
        epoch_error = 0.0
        temp_wX , temp_dX, temp_labels = generate_batch_pvdm(doc_ids=doc_ids, doc_repr=doc_repr)
        feed_dict = {train_wX : temp_wX, train_dX : temp_dX,train_label : temp_labels}
        op, l = sess.run([optimizer, loss], 
                                    feed_dict=feed_dict)
        
        epoch_error += l
                
        if step % 1000 == 0:
            print "Error at epoch : ", step, " = ", epoch_error
            
    save_path = saver.save(sess, "../../models/model_pvdm_batch_training.ckpt")
    print("Model saved in file: %s" % save_path)

Error at epoch :  0  =  0.91714411974
Error at epoch :  1000  =  0.170451268554
Error at epoch :  2000  =  0.0208349116147
Error at epoch :  3000  =  0.00401923665777
Error at epoch :  4000  =  0.00744339078665
Error at epoch :  5000  =  0.00306397257373
Error at epoch :  6000  =  0.00147702242248
Error at epoch :  7000  =  0.00204416946508
Error at epoch :  8000  =  0.00548148155212
Error at epoch :  9000  =  0.0045495540835
Error at epoch :  10000  =  0.00108600954991
Model saved in file: ../../models/model_pvdm_batch_training.ckpt


### Evaluation of the representation 

In [38]:
doc_pvdm = None

with tf.Session() as sess:
    saver = tf.train.Saver()
    # Restore variables from disk.
    saver.restore(sess, "../../models/model_pvdm_batch_training.ckpt")
    print("Model restored.")
    
    # Normalize word2vec 
    #norm_w = tf.sqrt(tf.reduce_sum(tf.square(word_embedding), 1, keep_dims=True))
    #normalized_word_embeddings = word_embedding / norm_w
    #word2vec = normalized_word_embeddings.eval()
    
    # Normalize doc2vec 
    #norm_d = tf.sqrt(tf.reduce_sum(tf.square(doc_embedding), 1, keep_dims=True))
    #normalized_doc_embeddings = doc_embedding / norm_d
    
    doc2vec = doc_embedding.eval()
    
    # Find performance 
    performanceTest(doc2vec, imdb_data['sentiment'], method=None)

Model restored.
('Train F1-score : ', 0.66742179431655402)
('Test F1-score : ', 0.66363757851129224)
